In [ ]:
import pandas as pd
import numpy as np

shark_df = pd.read_excel('./shark-dataset.xls')
# create dataframe copy
original_df = shark_df.copy()
shark_df

Data cleaning to-do's:
- (done) column names to lowercase
- (done) column names - remove empty space after name
- (done) column names - replace empty spaces with underscores
- find unique values in columns
- find duplicate rows
- year names without floating
- column 'Unnamed: 21' & 'Unnamed: 22' - what do we do with them?

In [ ]:
# column names to lowercase, remove empty space in the end, replace empty spaces with underscore
shark_df.columns = shark_df.columns.str.lower().str.strip().str.replace(" ", "_")

# remove multiple columns:
shark_df = shark_df.drop(['type', 'state', 'name', 'species', 'source', 'pdf', 'href_formula', 'href', 'case_number', 'case_number.1', 'original_order', 'unnamed:_21', 'unnamed:_22', 'time'], axis=1)

# convert year values to int
# shark_df['year'] = shark_df['year'].apply(lambda x: int(x) if isinstance(x, float) else x)
shark_df

TUNG: YEARS
---

In [ ]:
dft = shark_df

# Filter the DataFrame because years date back to the birth of jesus christ, our lord and saviour. 
start_year = 2014
end_year = 2024

dft = shark_df[(shark_df["year"] >= start_year) & (shark_df["year"] <= end_year)]

#convert float in year to int
#this way doesn't convert into int, but error messages disappears, WHYYYY
dft["year"] = dft["year"].fillna(0).astype(int)

#this conversion works, however the stupid error message 
#year_filtered_dft["year"] = year_filtered_dft["year"].apply(lambda x : int(x))

# showing that there are NO NaNs in date and year; 
print(dft.isnull().sum())
print("\nThe unique values in year are:\n", dft.year.unique())
print("\nThe datatype for column is currently:\n", dft.year.dtype)

dft

TUNG: DATES
---

In [ ]:
from datetime import datetime
import pandas as pd
import re



# Function to parse different date formats
def parse_date(date_str):
    if isinstance(date_str, str):
        try: 
            return pd.to_datetime(date_str)  # Try direct conversion
        except ValueError:
            match = re.search(r'(\d{4}-\d{1,2}-\d{1,2}|\d{1,2}-[A-Za-z]{3}-\d{4}|\b[A-Za-z]{3}-\d{4}\b)', date_str)
            if match:
                date_str = match.group(0)
                try:
                    return datetime.strptime(date_str, "%Y-%m-%d")
                except ValueError:
                    try:
                        return datetime.strptime(date_str, "%d-%b-%Y")
                    except ValueError:
                        try:
                            return datetime.strptime(date_str, "%b-%Y")
                        except ValueError:
                            return None  # Return None for invalid formats
    elif isinstance(date_str, datetime):
        return date_str  # Return the datetime object as is
    return None  # Return None if not a string or datetime

# Create datetime_column and string_column
dft["datetime_column"] = dft["date"].apply(parse_date)
dft["string_column"] = dft["date"].apply(lambda x: x if isinstance(x, str) else None)

# Drop rows with invalid datetime values
dft = dft[dft["datetime_column"].notna()]

# Extract month and year from datetime_column
dft['month'] = dft["datetime_column"].apply(lambda x: x.month if pd.notnull(x) else None)
dft['year'] = dft["datetime_column"].apply(lambda x: x.year if pd.notnull(x) else None)
#dft['month'] = dft["datetime_column"].dt.month
#dft['year'] = dft["datetime_column"].dt.year

# Define season mapping
season_mapping = {
    "Spring": [3, 4, 5],
    "Summer": [6, 7, 8],
    "Autumn": [9, 10, 11],
    "Winter": [12, 1, 2]
}

# Function to assign season based on month
def what_season(month):
    for season, months in season_mapping.items():
        if month in months:
            return season
    return None

# Assign season based on the extracted month
dft['season'] = dft['month'].apply(what_season)

# Check the resulting DataFrame
print(dft[['date', 'datetime_column', 'string_column', 'year', 'month', 'season']])




extras

groups_by_seanson_and_injury = dft.groupby(["season", "injury"])
groups_by_seanson_and_injury.get_group("Autumn")


# Country

In [ ]:
# Johanna - country and location
dfj = shark_df.copy()

# unique countries
dfj.country.unique() # many many countries (unsure values and very small countries)
top_20_countries = dfj.country.value_counts().head(20)

# stick to top 20 countries
dfj = dfj[dfj.country.isin(top_20_countries.index)] # deletes around 1000 of originally 6900 rows

# first letter in uppercase for all countries except USA for consistency
dfj.country = dfj.country.apply(lambda x: x.title() if x != 'USA' else x)
dfj.head()

In [ ]:
#drop location column
dfj_USA = dfj[dfj.country.isin(['USA'])]
dfj_USA.head()
dfj_USA.location.unique() # too complicated --> stick to USA overall

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x=top_20_countries.index.str.title(), y=top_20_countries.values, palette='viridis')
plt.title('Top 20 countries of shark attacks')
plt.xlabel('Country')
plt.tick_params(axis='x', labelsize=9)
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Jonathan - sex wrangling

#my copy of df
jon_df = shark_df.copy()

#check unique values
unique_values_sex = jon_df['sex'].value_counts(dropna=False)
print(f"Unique values before cleaning: {unique_values_sex}")

# Remove extra spaces
jon_df['sex'] = jon_df['sex'].str.strip()

# Replace specific values
jon_df['sex'] = jon_df['sex'].replace({
    'M': 'M',  # Standard "M"
    'F': 'F',  # Standard "F"
    'N': np.nan,  # Likely an error or placeholder, so set as NaN
    'M x 2': 'M',  # Assuming this means "male," so map to "M"
    'lli': np.nan,  # Unrecognized value, replace with NaN
    '.': np.nan,  # Unrecognized value, replace with NaN
    ' M': 'M'  # Correct extra space for "M"
})

jon_df['sex']= jon_df['sex'].fillna('unknown')

#replaced NaN as unknown for now? Suggestion: change the unknown values to M or F based on the percentage of M and F we do know. 
# eg. total M is 88% of total, and F is 12%. Apply those percentages to the unknown values.

# Step 1: Calculate the counts of "M" and "F"
#total_known = jon_df['sex'].value_counts()
#m_count = total_known['M']
#f_count = total_known['F']
#total = m_count + f_count

# Step 2: Calculate the percentages of "M" and "F"
#m_percentage = m_count / total
#f_percentage = f_count / total

# Step 3: Determine the number of "Unknown" values
#unknown_count = jon_df['sex'].value_counts().get('Unknown', 0)

# Step 4: Calculate how many "Unknown" values to fill with "M" and "F"
#m_fill_count = int(m_percentage * unknown_count)
#f_fill_count = unknown_count - m_fill_count  # Remaining to ensure the total matches

# Step 5: Replace "Unknown" values with "M" and "F" based on calculated counts
# First, get the indices of the "Unknown" entries
#unknown_indices = jon_df[jon_df['sex'] == 'Unknown'].index

# Randomly sample indices for M and F replacements
#m_indices = np.random.choice(unknown_indices, size=m_fill_count, replace=False)
#f_indices = unknown_indices.difference(m_indices)

# Assign "M" to sampled indices and "F" to remaining indices
#jon_df.loc[m_indices, 'sex'] = 'M'
#jon_df.loc[f_indices, 'sex'] = 'F'

jon_df['sex']= jon_df['sex'].fillna('unknown')


unique_values_sex = jon_df['sex'].value_counts(dropna=False)
print(f"Unique values after cleaning: {unique_values_sex}")



In [ ]:
# Jonathan - age wrangling

import numpy as np

# Set Pandas option to display all rows
pd.set_option('display.max_rows', None)

# Show all unique values in the 'age' column, including NaN counts
unique_values_age = jon_df['age'].value_counts(dropna=False)
print(f"Unique values before cleaning:\n{unique_values_age}")

# Reset the display option back to default to avoid affecting other output
pd.reset_option('display.max_rows')

In [ ]:
import re

# Step 1: Handle common descriptive terms with approximate ages
def convert_descriptive_age(value):
    if pd.isnull(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["teen", "teens"]:
        return 15  # Approximate age for teenagers
    elif value == "adult":
        return 30  # General average for adult age
    elif value in ["middle age", '"middle-age"']:
        return 45  # Approximate age for middle age
    elif value == "elderly":
        return 70  # Approximate age for elderly
    elif value in ["a minor", "young"]:
        return 10  # Assume a minor is around 10 years old
    elif value == "infant" or value == "9 months" or value == "2 to 3 months":
        return 1  # Age 1 for infants
    elif "month" in value:
        return 1  # Treat other month values as infants
    return value

jon_df['age'] = jon_df['age'].apply(convert_descriptive_age)

def convert_to_first_age(value):
    if isinstance(value, str):
        # Extract first number found in the string, ignoring the rest
        numbers = re.findall(r'\d+', value)
        if numbers:
            return int(numbers[0])  # Use the first number as the age
    return value

jon_df['age'] = jon_df['age'].apply(convert_to_first_age)

def convert_half_age(value):
    if isinstance(value, str) and "½" in value:
        # Replace "½" with ".5" and convert to float
        return float(value.replace("½", ".5"))
    return value  # Return unchanged if not fractional

jon_df['age'] = jon_df['age'].apply(convert_half_age)


def convert_irregular_entries(value):
    if isinstance(value, str):
        # If the string contains no digits at all, set it to NaN
        if not any(char.isdigit() for char in value):
            return np.nan
    return value

jon_df['age'] = jon_df['age'].apply(convert_irregular_entries)

jon_df['age'] = jon_df['age'].astype('Int64')

# Replace <NA> with a specific integer value
jon_df['age'] = jon_df['age'].fillna(0)  # Replaces <NA> with 0, for the sake of easier manipulation?

# Set Pandas option to display all rows
pd.set_option('display.max_rows', None)

# Show all unique values in the 'age' column, including NaN counts
unique_values_age = jon_df['age'].value_counts(dropna=False)
print(f"Unique values after cleaning:\n{unique_values_age}")

# Reset the display option back to default to avoid affecting other output
pd.reset_option('display.max_rows')



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

gender_counts = jon_df['sex'].value_counts()
print(gender_counts)

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='sex', data=jon_df, order=gender_counts.index)
plt.title('Shark Attacks by Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Attacks')
plt.show()

In [ ]:
age_bins = [0, 17, 30, 50, 100]  # Define bins for age ranges
age_labels = ['0-17', '18-30', '31-50', '51+']
jon_df['age_group'] = pd.cut(jon_df['age'], bins=age_bins, labels=age_labels)

age_group_counts = jon_df['age_group'].value_counts().sort_index()
print(age_group_counts)

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='age_group', data=jon_df, order=age_group_counts.index)
plt.title('Shark Attacks by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Number of Attacks')
plt.show()

In [ ]:
# Bru - fatality wrangling

#copy df for safety reasons
bru_df = shark_df.copy()

# rename column
bru_df.rename(columns={'unnamed:_11': 'fatal'}, inplace=True)

# check unique values
unique_values_fatal = bru_df['fatal'].unique()
print(f"Unique values before cleaning: {unique_values_fatal}")

# replace values
bru_df['fatal'] = bru_df['fatal'].replace({'N':'no','Y':'yes','n':'no','Y x 2':'yes',' N':'no','N ':'no','y':'yes','UNKNOWN':'unknown'})

# remove rows with specific values
rows_to_remove = ['Nq', 'M', 'F', 2017]
bru_df = bru_df[~bru_df['fatal'].isin(rows_to_remove)]

#drop rows with NaN values
bru_df = bru_df.dropna(subset=['fatal'])

# check again
unique_values_fatal = bru_df['fatal'].unique()
print(f"Unique values after cleaning: {unique_values_fatal}")

# print(bru_df.describe())
print(bru_df.groupby('fatal').count())

In [ ]:
# Bru - activity wrangling
from collections import Counter
import re

# check unique values
unique_values_activity = bru_df['activity'].unique()
# print(unique_values_activity)

# convert all values to a common case
bru_df['activity'] = bru_df['activity'].str.strip().str.lower().str.replace(r"[\"']", '', regex=True)

'''
The word_count() function uses collections and re to concatenate all the values in the column to a single string.
Then it will split the combined string into individual words.
Using Counters, it will return the frequency of each word.
It will also store the top 10 most common words in a list
'''

most_common_words = []

def word_count():
    bru_df['activity'] = bru_df['activity'].fillna('').astype(str) # replace NaN values with an empty string and convert all to string
    all_text = ' '.join(bru_df['activity']) # combine all values into a single string
    words = re.findall(r'\w+', all_text.lower()) # split into words (regex to handle punctuation)
    word_counts = Counter(words) # count word frequency
    most_common_words = [word for word, count in word_counts.most_common(50) if len(word) >= 5]
    return most_common_words
    return word_count

word_count()
most_common_words = word_count()

#print("Most common words")
#print(most_common_words)

print('Top values before replacement function\n', bru_df['activity'].value_counts().head(10))
#manually entered seoelcted values
selected_values_to_replace = ['surfing', 'diving', 'fishing', 'swimming', 'wading', 'bathing', 'snorkeling', 'kayaking', 'body boarding', 'scuba diving']


def replace_values():
    for word_to_replace in selected_values_to_replace:
        bru_df.loc[bru_df['activity'].str.contains(word_to_replace, case=False, na=False), 'activity'] = word_to_replace

    return bru_df

updated_bru_df = replace_values()

print('\nTop values after replacement:\n', updated_bru_df['activity'].value_counts().head(10))

# print("Word count:")
# print(word_counts_df)

In [ ]:
# playing with visual representation
import matplotlib.pyplot as plt
import seaborn as sns

# Count top 10 values in the 'activity' column
top_values = updated_bru_df['activity'].str.capitalize().value_counts().head(10)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x=top_values.index, y=top_values.values, palette='viridis')
plt.title('Top 10 activities that may result in shark attacks')
plt.xlabel('Activity')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Columns we need in the project
- Years
- date (months?)
- activity (can we stand on a beach?)
- country/state/location -> do we need all? let's start with country + location and see how messy it gets from then on?
- age - hospital costs might change
- sex
- unnamed 11 (fatal or not) yes for fatal
- injury (how does that combine with activity type?)
- Jonathan adds column with organs lost

### Formula
probably looks like : who are you, where are you, what are you going to do and when? -> probability calculated through that (will it be fatal or not), and then the result can be checked into the three insurance categories

- location is going to be an important parameter here (another correlation to definitely check, big influential factor?)

- !POINT SYSTEM!

- add point system for column unique values (genius)

### Other parameters to take into account:
- comparison between years to calculate potential drama in upcoming years
- calculate probability based on different parameters
- we're g etting creepy

## Next steps
1. Select columns we will use
2. Inspect column data, see where we have null or empty values
3. Decide what to do with the above
4. 'homogenize' column values
5. Figure out calculations for probability. 